In [1]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

In [2]:
tf.set_random_seed(321473584)

In [7]:
sentence = " But, in a larger sense, we can not dedicate, we can not consecrate, we can not hallow this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for us to be here dedicated to the great task remaining before us—that from these honored dead we take increased devotion to that cause for which they gave the last full measure of devotion—that we here highly resolve that these dead shall not have died in vain—that this nation, under God, shall have a new birth of freedom—and that government of the people, by the people, for the people, shall not perish from the earth"

In [8]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

In [11]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 20  # Any arbitrary number
learning_rate = 0.1

In [12]:
dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

0  But, in a larger se -> But, in a larger sen
1 But, in a larger sen -> ut, in a larger sens
2 ut, in a larger sens -> t, in a larger sense
3 t, in a larger sense -> , in a larger sense,
4 , in a larger sense, ->  in a larger sense, 
5  in a larger sense,  -> in a larger sense, w
6 in a larger sense, w -> n a larger sense, we
7 n a larger sense, we ->  a larger sense, we 
8  a larger sense, we  -> a larger sense, we c
9 a larger sense, we c ->  larger sense, we ca
10  larger sense, we ca -> larger sense, we can
11 larger sense, we can -> arger sense, we can 
12 arger sense, we can  -> rger sense, we can n
13 rger sense, we can n -> ger sense, we can no
14 ger sense, we can no -> er sense, we can not
15 er sense, we can not -> r sense, we can not 
16 r sense, we can not  ->  sense, we can not d
17  sense, we can not d -> sense, we can not de
18 sense, we can not de -> ense, we can not ded
19 ense, we can not ded -> nse, we can not dedi
20 nse, we can not dedi -> se, we can not dedic
21

309  they did here. It i -> they did here. It is
310 they did here. It is -> hey did here. It is 
311 hey did here. It is  -> ey did here. It is f
312 ey did here. It is f -> y did here. It is fo
313 y did here. It is fo ->  did here. It is for
314  did here. It is for -> did here. It is for 
315 did here. It is for  -> id here. It is for u
316 id here. It is for u -> d here. It is for us
317 d here. It is for us ->  here. It is for us 
318  here. It is for us  -> here. It is for us t
319 here. It is for us t -> ere. It is for us th
320 ere. It is for us th -> re. It is for us the
321 re. It is for us the -> e. It is for us the 
322 e. It is for us the  -> . It is for us the l
323 . It is for us the l ->  It is for us the li
324  It is for us the li -> It is for us the liv
325 It is for us the liv -> t is for us the livi
326 t is for us the livi ->  is for us the livin
327  is for us the livin -> is for us the living
328 is for us the living -> s for us the living,
329 s for us the liv

571 e take increased dev ->  take increased devo
572  take increased devo -> take increased devot
573 take increased devot -> ake increased devoti
574 ake increased devoti -> ke increased devotio
575 ke increased devotio -> e increased devotion
576 e increased devotion ->  increased devotion 
577  increased devotion  -> increased devotion t
578 increased devotion t -> ncreased devotion to
579 ncreased devotion to -> creased devotion to 
580 creased devotion to  -> reased devotion to t
581 reased devotion to t -> eased devotion to th
582 eased devotion to th -> ased devotion to tha
583 ased devotion to tha -> sed devotion to that
584 sed devotion to that -> ed devotion to that 
585 ed devotion to that  -> d devotion to that c
586 d devotion to that c ->  devotion to that ca
587  devotion to that ca -> devotion to that cau
588 devotion to that cau -> evotion to that caus
589 evotion to that caus -> votion to that cause
590 votion to that cause -> otion to that cause 
591 otion to that ca

792 freedom—and that gov -> reedom—and that gove
793 reedom—and that gove -> eedom—and that gover
794 eedom—and that gover -> edom—and that govern
795 edom—and that govern -> dom—and that governm
796 dom—and that governm -> om—and that governme
797 om—and that governme -> m—and that governmen
798 m—and that governmen -> —and that government
799 —and that government -> and that government 
800 and that government  -> nd that government o
801 nd that government o -> d that government of
802 d that government of ->  that government of 
803  that government of  -> that government of t
804 that government of t -> hat government of th
805 hat government of th -> at government of the
806 at government of the -> t government of the 
807 t government of the  ->  government of the p
808  government of the p -> government of the pe
809 government of the pe -> overnment of the peo
810 overnment of the peo -> vernment of the peop
811 vernment of the peop -> ernment of the peopl
812 ernment of the p

In [13]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

In [14]:
# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape

Tensor("one_hot:0", shape=(?, 20, 30), dtype=float32)


In [23]:
# def lstm_cell():
#     #출력(hidden_size)을 얼마나 할 것인지 정하는 것이 중요
#     cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
#     cell = rnn.MultiRNNCell([cell]*3,state_is_tuple=True)
#     #층을 더 쌓음
#     return cell

In [25]:
#multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True) 
multi_cells = rnn.MultiRNNCell([cell]*3,state_is_tuple=True)

In [17]:
# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

In [28]:
X_for_softmax = tf.reshape(outputs,[-1,hidden_size])
softmax_w = tf.get_variable("softmax_w",[hidden_size,num_classes])
softmax_b = tf.get_variable("softmax_b",[num_classes])
outputs = tf.matmul(X_for_softmax,softmax_w)+softmax_b
outputs=tf.reshape(outputs,[batch_size,sequence_length,num_classes])

In [29]:
# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [30]:
for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)


0 0 bbbbbbbbbbbbbbiiiiii 3.40527
0 1 bbbbbbbbbbb——iiiiiii 3.40527
0 2 bbbbbbbbbbbiiiiiiiii 3.40527
0 3 bbbbbbbbb—iiiiiiiiib 3.40527
0 4 bbbbbbbb——iiiiiiiibb 3.40527
0 5 bbbbbb———iiiiiiiibbb 3.40527
0 6 bbbbb———iiiiiiiibbbb 3.40527
0 7 bbbi———iiiiiiiiibbbb 3.40527
0 8 bbiiiiiiiiiiiiibbbbb 3.40527
0 9 bbb——iiiiiiiiibbbbbb 3.40527
0 10 bbbiiiiiiiiibbbbbbbb 3.40527
0 11 bbbiiiiiiiiibbbbbbbb 3.40527
0 12 bbiiiiiiiibbbbbbbbbb 3.40527
0 13 bbbiiiiibbbbbbbbbbbb 3.40527
0 14 bbbbbbbbbbbbbbbbbbbb 3.40527
0 15 bbiiibbbbbbbbbbbbbbb 3.40527
0 16 biibbbbbbbbbbbbbbbbb 3.40527
0 17 bbbbbbbbbbbbbbbbbbbb 3.40527
0 18 bbbbbbbbbbbbbbbbbbbb 3.40527
0 19 bbbbbbbbbbbbbbbbbbbb 3.40527
0 20 bbbbbbbbbbbbbbbbbbbb 3.40527
0 21 bbbbbbbbbbbbbbbbbbbb 3.40527
0 22 bbbbbiiiiiiiiibbbbbb 3.40527
0 23 bbbbbbbbiiiiibbbbbbb 3.40527
0 24 bbbibbbiiiibbbbbbbbb 3.40527
0 25 bbbbbbi—iiiibbbbbbbb 3.40527
0 26 bbbbbbiiiibbbbbbbbbb 3.40527
0 27 bbbbb—bbbbbbbbbbbbbb 3.40527
0 28 bbbb—bbbbbbbbbbbbbbb 3.40527
0 29 bbb—iiiibbbbbbbbbbb

0 276 bbbbiiiiiiibbbbbbbbb 3.40527
0 277 bbbiibbibbbbbbbbbbbb 3.40527
0 278 bbbbbbbbbbbbbbbbbbbb 3.40527
0 279 bbbbbbbbbbbbbbbbbbbb 3.40527
0 280 bbbbbbbbbbbbbbbbbbbi 3.40527
0 281 bbbbbbbbbbbbbbbbbiii 3.40527
0 282 bbbbbbbbbbbbbbbbiiii 3.40527
0 283 bbbbbbbbbbbbbbbiiiii 3.40527
0 284 bbbbbbbbbbbbbbiiiiii 3.40527
0 285 bbbbbbbbbbbbbiiiiiii 3.40527
0 286 bbbbbbbbbbbiiiiiiiii 3.40527
0 287 bbbbb—bbbbiiiiiiiiii 3.40527
0 288 bbbb—bbbbiiiiiiiiiii 3.40527
0 289 bbb—iiiiiiiiiiiiiiii 3.40527
0 290 bbbbbbbbiiiiiiiiiiii 3.40527
0 291 bbbbbbiiiiiiiiiiiiii 3.40527
0 292 bbbbbbiiiiiiiiiiiiii 3.40527
0 293 bbbbbiiiiiiiiiiiiiii 3.40527
0 294 bbbbiiiiiiiiiiiiiiii 3.40527
0 295 bbiiiiiiiiiiiiiiiiii 3.40527
0 296 biiiiiiiiiiiiiiiiiib 3.40527
0 297 bbbbbiiiiiiiiiiiiibb 3.40527
0 298 bbbbbiiiiiiiiiiiiibb 3.40527
0 299 bbbbbiiiiiiiiiiibbbb 3.40527
0 300 bbbbiiiiiiiiiiibbbbb 3.40527
0 301 bbbbbiiiiiiiiibbbbbb 3.40527
0 302 bbbbiiiiiiiiibbbbbbb 3.40527
0 303 bbbiiiiiiiiibbbbbbbb 3.40527
0 304 bbiiiiiiiiibbb

0 515 bbbbbbbbiiiiibbbbbbb 3.40527
0 516 bbbbbbbiiiiiibbbbbbb 3.40527
0 517 bbbbbbiiiiiibbbbbbbb 3.40527
0 518 bbbbbbiiibbbbbbbbbbi 3.40527
0 519 bbbiiiiiiibbbbbbbbii 3.40527
0 520 bbiiiiiiiibbbbbbbiii 3.40527
0 521 bbiiiiiiiibbbbbiiiii 3.40527
0 522 bbiiii———bbbbbbiiibb 3.40527
0 523 biii————bbbbbbiiibbb 3.40527
0 524 bbbb—bbbbbbbbiiibbbb 3.40527
0 525 bbbbbbbbbbbbbbbbbbbb 3.40527
0 526 bbbbbbbbbbbbibbbbbbb 3.40527
0 527 bbbbbbbbbbiibbbbbbbb 3.40527
0 528 bbbbbbbbbiiibbbbbbbb 3.40527
0 529 bbbbbbbbiiibbbbbbbbi 3.40527
0 530 bbbbbbiiiiibbbbbbiii 3.40527
0 531 bbbbbbiiiiibbbbbiiii 3.40527
0 532 bbbbbiiiiibbbbbiiiii 3.40527
0 533 bbbbiiiibbbbbbiiiiii 3.40527
0 534 bbbiibbbbbbbbbiiiiii 3.40527
0 535 bbiibbbbbbbbbiiiiiii 3.40527
0 536 bbbbbbbbbbbbbiiiiiii 3.40527
0 537 bbbbbbbbbbbbiiiiiiib 3.40527
0 538 bbbbbbbbbbbiiiiiiibb 3.40527
0 539 bbbbbbbbbbiiiiiiibbb 3.40527
0 540 bbbbbbiiiiiiiiiiiibb 3.40527
0 541 bbbbiiiiiiiiiiiiibbb 3.40527
0 542 bbbiiiiiiiiiiiiibbbb 3.40527
0 543 bbiiiiiiiiiiii

1 21                      3.05679
1 22                      3.05679
1 23                      3.05679
1 24                      3.05679
1 25                      3.05679
1 26                      3.05679
1 27                      3.05679
1 28                      3.05679
1 29                      3.05679
1 30                      3.05679
1 31                      3.05679
1 32                      3.05679
1 33                      3.05679
1 34                      3.05679
1 35                      3.05679
1 36                      3.05679
1 37                      3.05679
1 38                      3.05679
1 39                      3.05679
1 40                      3.05679
1 41                      3.05679
1 42                      3.05679
1 43                      3.05679
1 44                      3.05679
1 45                      3.05679
1 46                      3.05679
1 47                      3.05679
1 48                      3.05679
1 49                      3.05679
1 50          

1 336                      3.05679
1 337                      3.05679
1 338                      3.05679
1 339                      3.05679
1 340                      3.05679
1 341                      3.05679
1 342                      3.05679
1 343                      3.05679
1 344                      3.05679
1 345                      3.05679
1 346                      3.05679
1 347                      3.05679
1 348                      3.05679
1 349                      3.05679
1 350                      3.05679
1 351                      3.05679
1 352                      3.05679
1 353                      3.05679
1 354                      3.05679
1 355                      3.05679
1 356                      3.05679
1 357                      3.05679
1 358                      3.05679
1 359                      3.05679
1 360                      3.05679
1 361                      3.05679
1 362                      3.05679
1 363                      3.05679
1 364               

...

2 384                      3.13783
2 385                      3.13783
2 386                      3.13783
2 387                      3.13783
2 388                      3.13783
2 389                      3.13783
2 390                      3.13783
2 391                      3.13783
2 392                      3.13783
2 393                      3.13783
2 394                      3.13783
2 395                      3.13783
2 396                      3.13783
2 397                      3.13783
2 398                      3.13783
2 399                      3.13783
2 400                      3.13783
2 401                      3.13783
2 402                      3.13783
2 403                      3.13783
2 404                      3.13783
2 405                      3.13783
2 406                      3.13783
2 407                      3.13783
2 408                      3.13783
2 409                      3.13783
2 410                      3.13783
2 411                      3.13783
2 412               

3 13                      2.93856
3 14                      2.93856
3 15                      2.93856
3 16                      2.93856
3 17                      2.93856
3 18                      2.93856
3 19                      2.93856
3 20                      2.93856
3 21                      2.93856
3 22                      2.93856
3 23                      2.93856
3 24                      2.93856
3 25                      2.93856
3 26                      2.93856
3 27                      2.93856
3 28                      2.93856
3 29                      2.93856
3 30                      2.93856
3 31                      2.93856
3 32                      2.93856
3 33                      2.93856
3 34                      2.93856
3 35                      2.93856
3 36                      2.93856
3 37                      2.93856
3 38                      2.93856
3 39                      2.93856
3 40                      2.93856
3 41                      2.93856
3 42          

3 280                      2.93856
3 281                      2.93856
3 282                      2.93856
3 283                      2.93856
3 284                      2.93856
3 285                      2.93856
3 286                      2.93856
3 287                      2.93856
3 288                      2.93856
3 289                      2.93856
3 290                      2.93856
3 291                      2.93856
3 292                      2.93856
3 293                      2.93856
3 294                      2.93856
3 295                      2.93856
3 296                      2.93856
3 297                      2.93856
3 298                      2.93856
3 299                      2.93856
3 300                      2.93856
3 301                      2.93856
3 302                      2.93856
3 303                      2.93856
3 304                      2.93856
3 305                      2.93856
3 306                      2.93856
3 307                      2.93856
3 308               

3 519                      2.93856
3 520                      2.93856
3 521                      2.93856
3 522                      2.93856
3 523                      2.93856
3 524                      2.93856
3 525                      2.93856
3 526                      2.93856
3 527                      2.93856
3 528                      2.93856
3 529                      2.93856
3 530                      2.93856
3 531                      2.93856
3 532                      2.93856
3 533                      2.93856
3 534                      2.93856
3 535                      2.93856
3 536                      2.93856
3 537                      2.93856
3 538                      2.93856
3 539                      2.93856
3 540                      2.93856
3 541                      2.93856
3 542                      2.93856
3 543                      2.93856
3 544                      2.93856
3 545                      2.93856
3 546                      2.93856
3 547               

KeyboardInterrupt: 

In [ ]:
# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:

        print(char_set[index[-1]], end='')

# 잘 안돼 
1)층 depp
2)넓이 wide

In [31]:
df

NameError: name 'df' is not defined